# Разработка проекта Blockchain

1. Для разработки проекта нам потребуется импортировать следующие библиотеки Python: 

In [ ]:
# Ваш код
import hashlib
import random
import json
import binascii
import pandas as pd
import numpy as np
import pylab as pl
import logging 
import datetime
import collections

2. В дополнение к указанным выше стандартным библиотекам мы собираемся подписать наши транзакции, создать хэш объектов. Для этого вам потребуется импортировать следующие библиотеки:

In [ ]:
!pip install crypto

     |████████████████████████████████| 593kB 5.7MB/s 


In [ ]:
!pip install pycrypto

     |████████████████████████████████| 450kB 5.4MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499931 sha256=983257721e25690fc0bedd6d185f3c799cbfec1af91f551ce9113888d0680e6d
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built pycrypto


In [ ]:
 # Ваш код
import Crypto
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

класс **Block**



Будем предполагать, что блок состоит из фиксированного числа транзакций, которое в данном случае равно трем. Поскольку блок должен хранить список этих трех транзакций, мы объявим переменную экземпляра под названием **verify_transactions** следующим образом:

self.verified_transactions = []

Мы назвали эту переменную как **Verified_transactions**, чтобы указать, что только проверенные действительные транзакции будут добавлены в блок. Каждый блок также содержит хэш-значение предыдущего блока, поэтому цепочка блоков становится неизменной.

Чтобы сохранить предыдущий хеш, мы объявляем переменную экземпляра следующим образом:

self.previous_block_hash = ""

**Nonce** для хранения одноразовых номеров, созданных майнером в процессе майнинга.

self.Nonce = ""

Полное определение класса **Block** приведено ниже:

In [ ]:
# Ваш код
class Block:
  def _init_(self):
    self.verifield_transaction=[]
    self.previous_block_hasg=""
    self.Nonce=""

Поскольку каждому блоку требуется значение хеша предыдущего блока, мы объявляем глобальную переменную с именем **last_block_hash** следующим образом:

In [ ]:
last_block_hash = ""

Класс **Client**

In [ ]:
# Ваш код
class Client:
  def __init__(self):
    random = Crypto.Random.new().read
    self._private_key=RSA.generate(1024,random) #закрытый
    self._public_key = self._private_key.publickey()#открытый который от закрытого берется
    self._signer = PKCS1_v1_5.new(self._private_key)

  @property
  def identity(self):
    return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

Клиент тестирования

In [ ]:
Dinesh = Client()
print (Dinesh.identity)

30819f300d06092a864886f70d010101050003818d0030818902818100d380d24db0398bd6563be17d4b276fe4754c5df34fca241d7835ac761ec33fd9f1dac1e85b19d74fe6e75385a965eaed08339de96acce60c4f9ae77863ba858eb8b65e27d9dce72e767fcbbfe28324331f93132d8b9a5f38e7fe045ae9b6f6c28194123e0f4784a50fe18e1db60f97f240f3718344ad5b0704cc3109b17145a10203010001


# Транзакции между клиентами

Класс **Transaction**

создадим класс Transaction, чтобы клиент мог кому-то отправлять деньги. Обратите внимание, что клиент может быть как отправителем, так и получателем денег. Когда вы хотите получить деньги, другой отправитель создаст транзакцию и укажет в ней ваш публичный адрес.

3. Инициализация класса транзакции:

In [ ]:
# Ваш код
class Transaction:
  def __init__(self, sender, recipient, value):
    self.sender=sender
    self.recipient=recipient
    self.value=value
    self.time=datetime.datetime.now()

  def to_dict(self):
    if self.sender=="Genesis":
      identity="Genesis"
    else:
      identity=self.sender.identity
    return collections.OrderedDict({
      'sender':identity,
      'recipient':self.recipient,
      'value':self.value,
      'time': self.time})

  def sign_transaction(self):
    private_key=self.sender._private_key
    signer=PKCS1_v1_5.new(private_key)
    h=SHA.new(str(self.to_dict()).encode('utf8'))
    return binascii.hexlify(signer.sign(h)).decode('ascii')


Метод init принимает три параметра — **открытый ключ** отправителя, **открытый ключ** получателя и **отправляемую сумму**. Они хранятся в переменных экземпляра для использования другими методами. Кроме того, мы создаем еще одну переменную для хранения **времени транзакции**.

**Класс тестовой транзакции**

Для этого мы создадим двух пользователей, называемых Динеш и Рамеш . Динеш отправит 5 TPCoins Рамешу. Для этого сначала мы создаем клиентов с именами Dinesh и Ramesh.

In [ ]:
Dinesh = Client()
Ramesh = Client()

при создании экземпляра класса Client будут созданы открытый и закрытый ключи, уникальные для клиента. Поскольку Динеш отправляет платеж Рамешу, ему понадобится открытый ключ Рамеша, полученный с помощью свойства идентификатора клиента.

Таким образом, мы создадим экземпляр транзакции, используя следующий код 

In [ ]:
t = Transaction(
   Dinesh,
   Ramesh.identity,
   5.0
)

первым параметром является отправитель, вторым параметром является открытый ключ получателя, а третьим параметром является сумма, подлежащая передаче. Метод sign_transaction извлекает закрытый ключ отправителя из первого параметра для подписания транзакции.

После создания объекта транзакции вы подпишите его, вызвав его метод sign_transaction . Этот метод возвращает сгенерированную подпись в формате для печати. Мы генерируем и печатаем подпись, используя следующие две строки кода:

In [ ]:
signature = t.sign_transaction()
print (signature) 

13554940463588f3ea0e551cd21749ca2fb82c9acd76b276e7c92fc8c85126f672796b8f10839f29e52225ae809b2ea61540e8b5fad257478e7cfa72485c9859a19e39b7f85287dd9a20c9a430296db46083204be889b64a8b945fabd6543523fa947a75dac9ccdcce50f181044668feaebd3dd9319ba2b2c0d2e839b10c32a7


 базовая инфраструктура создания клиента и транзакции готова, у нас теперь будет несколько клиентов, выполняющих несколько транзакций, как в реальной жизни.

# Создание блока Genesis

Мы предполагаем, что создатель TPCoins изначально выдает 500 TPCoins известному клиенту Dinesh . Для этого он сначала создает экземпляр Dinesh 

In [ ]:
Dinesh = Client()

In [ ]:
t0 = Transaction (
   "Genesis",
   Dinesh.identity,
   500.0
)

Теперь мы создаем экземпляр класса Block и называем его block0 .

In [ ]:
block0 = Block()

На этом этапе блок полностью инициализирован и готов к добавлению в нашу цепочку блоков. Мы будем создавать блокчейн для этой цели. Перед тем, как добавить блок в цепочку блоков, мы хешируем блок и сохраняем его значение в глобальной переменной last_block_hash, которую мы объявили ранее. Это значение будет использовано следующим майнером в его блоке.

Мы используем следующие две строки кода для хеширования блока и сохранения значения дайджеста.

In [ ]:
digest = hash (block0)
last_block_hash = digest

Мы инициализируем переменные экземпляра previous_block_hash и Nonce как None , поскольку это самая первая транзакция, которая будет сохранена в нашей цепочке блоков.

In [ ]:
block0.previous_block_hash = None
Nonce = None

Далее мы добавим вышеупомянутую транзакцию t0 в список Verified_transactions, поддерживаемый в блоке

In [ ]:
block0.verified_transactions.append (t0)

# Создание нескольких транзакций

Сделки, совершенные различными клиентами, ставятся в очередь в системе; 

майнеры забирают транзакции из этой очереди и добавляют их в блок. 

Затем они будут добывать блок, и победивший майнер получит привилегию добавить блок в цепочку блоков и тем самым заработать немного денег для себя.

Добавим небольшую служебную функцию для печати содержимого данной транзакции.

1.   **Просмотр транзакции**

Функция **display_transaction** принимает один параметр типа транзакции. Объект словаря в полученной транзакции копируется во временную переменную с именем **dict**, и с помощью ключей словаря различные значения выводятся на консоль.

In [ ]:
# Ваш код

2.   **Очередь транзакций**


Чтобы создать очередь, мы объявляем переменную глобального списка под названием транзакции следующим образом:

In [ ]:
transactions = []

3.   **Создание нескольких клиентов**

Теперь мы начнем создавать транзакции. Сначала мы создадим четырех клиентов, которые будут отправлять друг другу деньги для получения различных услуг или товаров от других.

In [ ]:
Dinesh = Client()
Ramesh = Client()
Seema = Client()
Vijay = Client()

На данный момент у нас есть четыре клиента: **Динеш, Рамеш, Сима и Виджай**. В настоящее время мы предполагаем, что у каждого из этих клиентов есть несколько **TPCoin** в их кошельках для проведения транзакций. Идентификационные данные каждого из этих клиентов будут определены с помощью свойства **identity** этих объектов.

4.   **Создание первой транзакции**

In [ ]:
t1 = Transaction(
   Dinesh,
   Ramesh.identity,
   15.0
)

В этой транзакции Динеш отправляет 5 **TPCoins** Рамешу. Чтобы транзакция была успешной, мы должны убедиться, что у Динеша в кошельке достаточно денег для этого платежа. Обратите внимание, что нам потребуется транзакция генезиса, чтобы запустить циркуляцию **TPCoin** в системе. 

Мы подпишем эту транзакцию, используя закрытый ключ Динеша, и добавим ее в очередь транзакций следующим образом:

In [ ]:
t1.sign_transaction()
transactions.append(t1)

5.   **Добавление дополнительных транзакций**

Теперь мы создадим еще несколько транзакций, каждая транзакция выдаст несколько TPCoins другой стороне. Когда кто-то тратит деньги, нет необходимости, чтобы он проверял наличие достаточных остатков в этом кошельке. В любом случае майнер будет проверять каждую транзакцию на баланс, который имеет отправитель при инициации транзакции.

В случае недостаточного баланса майнер пометит эту транзакцию как недействительную и не добавит ее в этот блок.

Следующий код создает и добавляет еще девять транзакций в нашу очередь.

In [ ]:
# Ваш код

Когда вы запустите приведенный выше код, у вас будет десять транзакций в очереди, чтобы майнеры могли создавать свои блоки.

6.   **Демпинг транзакций**

Как менеджер блокчейна, вы можете периодически просматривать содержимое очереди транзакций. Для этой цели вы можете использовать функцию **display_transaction**, которую мы разработали ранее. Чтобы сбросить все транзакции в очереди, просто выполните итерацию списка транзакций и для каждой указанной транзакции вызовите функцию **display_transaction**, как показано здесь:

In [ ]:
for transaction in transactions:
   display_transaction (transaction)
   print ('--------------')

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100d290540a8ea0ccd0675c1de6179c5d193907ef9bb17a3d1daaf39ade89fa1acc367ce73be6030ae6613de61f6798316eab73a977a2ae5b84fac5c7066c39436c05700e3fd282ed30c04a5cc71aade53beba4926fd1eabcf49a1049831f63ed7b19763a8da9aab7e57fb3b54e536ba2c06edce38eb09e59ccdc7ed812900370db0203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100be8d5bd9c12511d89eb1d0b7163ec3b392a5878cadb869164a5305f0ce979849272a2bc165109cd234b65f960654ae876cb2a5e6846fd09271a300fb551ef0d474be52f5501f8c34903a12a0ac039011af8435147695519bc6e4f1b67d0fa6f4a70337c9d7fad750e359d5bf84f491acd8e712b97576c2f716eed00d629fe3250203010001
-----
value: 15.0
-----
time: 2020-10-03 09:05:22.442746
-----
--------------


До сих пор вы узнали, как создавать клиентов, разрешать их между собой и поддерживать очередь ожидающих транзакций, которые должны быть добыты. Теперь самая важная  — это создание самой блокчейна.

**Добавление первого блока**

Чтобы добавить новый блок, мы сначала создаем экземпляр класса **Block** .

In [ ]:
block = Block()

In [ ]:
t = Transaction(Dinesh, Ramesh.identity,5.0)

In [ ]:
Dinesh = Client()
print (Dinesh.identity)

30819f300d06092a864886f70d010101050003818d0030818902818100b7aad5ad38f51e0b98742714fb74af5a029f18e1d51c3b2cd60dae4dbd94b3a0c37e9af79bd4017dceb1194bc2adcb73441852d75726a558321190b54fec0de7bcdd8fc80d202044352eb90e28b02b77fa822cc17977a44e2f70c08f345b7600df9380aa28d08c52f830cdfcdefd3a6200ecbe72c019512bf75675cc2e33c2390203010001


# **Создание блокчейна**

Блокчейн содержит список блоков, связанных друг с другом. Чтобы сохранить весь список, мы создадим переменную списка с именем **TPCoins** 

In [ ]:
TPCoins = []

напишем служебный метод dump_blockchain для выгрузки содержимого всей цепочки блоков. Сначала мы печатаем длину блокчейна, чтобы узнать, сколько блоков в настоящий момент присутствует в блокчейне.

In [ ]:
def dump_blockchain (self):
   print ("Number of blocks in the chain: " + str(len (self)))

с течением времени количество блоков в цепочке блоков будет чрезвычайно большим для печати. Таким образом, когда вы печатаете содержимое блокчейна, вам может потребоваться выбрать диапазон, который вы хотели бы изучить. В приведенном ниже коде мы напечатали весь блокчейн, так как мы не добавляем слишком много блоков в текущей демонстрации.

In [ ]:
for x in range (len(TPCoins)):
   block_temp = TPCoins[x]
   print ("block # " + str(x))

Каждый указанный блок копируется во временную переменную с именем **block_temp**.

Мы печатаем номер блока как заголовок для каждого блока. Обратите внимание, что числа будут начинаться с нуля, первый блок — это блок генеза, который нумеруется с нуля.

В каждом блоке мы сохранили список из трех транзакций (за исключением блока генезиса) в переменной с именем Verified_transactions . Мы повторяем этот список в цикле for, и для каждого полученного элемента мы вызываем функцию display_transaction, чтобы отобразить детали транзакции.

In [ ]:
# Ваш код

# **Добавление блока Genesis**

Добавление блока в цепочку блоков включает добавление созданного блока в наш список **TPCoins**.

In [ ]:
# Ваш код

в отличие от остальных блоков в системе, блок генезиса содержит только одну транзакцию, инициированную инициатором системы TPCoins. Теперь вы сбросите содержимое блокчейна, вызвав нашу глобальную функцию dump_blockchain

In [ ]:
# Ваш код

На данный момент система блокчейна готова к использованию. Теперь мы дадим возможность заинтересованным клиентам стать майнерами, предоставив им функциональность майнинга.

# **Создание майнеров**

Для включения майнинга нам нужно разработать функцию майнинга. Функциональность майнинга должна генерировать дайджест по заданной строке сообщения и предоставлять подтверждение работы.

1.   **Функция дайджеста сообщения**

Мы напишем служебную функцию sha256 для создания дайджеста по данному сообщению 

In [ ]:
def sha256(message):
  return hashlib.sha256(message.encode('ascii')).hexdigest()

**Функция майнинга**

Разработаем **функцию майнинга**, которая реализует нашу собственную стратегию майнинга. 

В этом случае наша стратегия заключается в создании хэша для данного сообщения с префиксом с указанным числом 1. Заданное количество единиц указано в качестве параметра для моей функции, указанной в качестве уровня сложности.

Например, если вы укажете уровень сложности 2, сгенерированный хэш для данного сообщения должен начинаться с двух 1 — например, 11xxxxxxxx. Если уровень сложности равен 3, сгенерированный хэш должен начинаться с трех 1 — как 111xxxxxxxx. Учитывая эти требования, мы теперь разработаем **функцию майнинга**, как показано в шагах, приведенных ниже.

**Шаг 1**

Функция майнинга принимает два параметра — сообщение и уровень сложности.


def mine(message, difficulty=1):

**Шаг 2**

Уровень сложности должен быть больше или равен 1, мы обеспечиваем это следующим утверждением assert:

assert difficulty >= 1

**Шаг 3**

создаем префиксную переменную, используя заданный уровень сложности.


prefix = '1' * difficulty

если уровень сложности равен 2, префикс будет «11», а если уровень сложности равен 3, префикс будет «111» и т. Д. Мы проверим, существует ли этот префикс в сгенерированном дайджесте сообщения. Чтобы переварить само сообщение, мы используем следующие две строки кода:

for i in range(1000):
   digest = sha256(str(hash(message)) + str(i))

продолжаем добавлять новый номер i в хэш сообщения в каждой итерации и генерируем новый дайджест по объединенному сообщению. Поскольку входные данные для функции sha256 меняются на каждой итерации, значение дайджеста также изменяется. Мы проверяем, имеет ли это значение дайджеста установленный префикс .

if digest.startswith(prefix):

Если условие выполнено, мы завершаем цикл for и возвращаем вызывающему значение дайджеста .

Весь мой :


In [ ]:
# Ваш код

оператор print печатает дайджест-значение и количество итераций, необходимых для выполнения условия перед возвратом из функции.

**Тестирование функции майнинга**

проверить  функцию майнинга, просто выполните:

In [ ]:
mine ("test message", 2)

'721e8681f5e94b05ca8d6afbbdcdc47fa216fc503c32f755c670f6222896b1c7'

# **Добавление первого блока**

Чтобы добавить новый блок, мы сначала создаем экземпляр класса **Block** .

In [ ]:
# Ваш код

Мы забираем первые 3 транзакции из очереди 

In [ ]:
# Ваш код